## **Preparing the dataset and Training the model**

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.6 torch-2.0.1+cu118 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.5/107.7 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
!mkdir "D:\quest_park" # Desired project location
%cd "D:\quest_park"

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="tUZCeeGNLljUnpXXsngy")
project = rf.workspace("swastik-abhijit-patra-zlamm").project("real-time-parking-lot-vehicle-detection")
dataset = project.version(1).download("yolov8")

mkdir: cannot create directory ‘D:\\quest_park’: File exists
/content/D:\quest_park
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.134`

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8n.yaml')  # build from YAML and transfer weights

# Train the model
model.train(data='coco128.yaml', epochs=30, imgsz=640) # increase the number epochs inorder to get more accuracy



                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

### **Final detection of the parking spaces**

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
import numpy as np
import cv2


In [ ]:
camera = cv2.VideoCapture("parking1.mp4")

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
model=YOLO(r'yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 122MB/s]



In [ ]:
file = open("/content/slot_coordinates.txt.txt")

print("[INFO] Loading parking coordinates ...")

lines = file.readlines()

lines = [line.strip() for line in lines]

total_parking_lots = len(lines)

parking_lot_coords = list()

for i in range(len(lines)):

    coords = lines[i].split()
    top_left = (int(coords[0]), int(coords[1]))
    top_right = (int(coords[2]), int(coords[3]))
    bottom_right = (int(coords[4]), int(coords[5]))
    bottom_left = (int(coords[6]), int(coords[7]))
    coord = np.array([top_left,top_right, bottom_right,bottom_left])
    parking_lot_coords.append(coord)
print(parking_lot_coords)

[INFO] Loading parking coordinates ...
[array([[ 53, 438],
       [133, 441],
       [105, 571],
       [ 23, 568]]), array([[109, 443],
       [205, 436],
       [187, 570],
       [ 89, 567]]), array([[181, 429],
       [287, 428],
       [277, 561],
       [167, 569]]), array([[246, 436],
       [350, 426],
       [365, 557],
       [259, 560]]), array([[331, 424],
       [424, 419],
       [455, 550],
       [356, 564]]), array([[406, 420],
       [497, 410],
       [545, 534],
       [446, 548]]), array([[475, 418],
       [579, 408],
       [640, 524],
       [530, 545]]), array([[540, 405],
       [646, 395],
       [723, 518],
       [612, 533]]), array([[605, 394],
       [715, 388],
       [801, 502],
       [691, 522]]), array([[674, 398],
       [776, 388],
       [877, 487],
       [767, 511]]), array([[737, 385],
       [831, 374],
       [931, 492],
       [859, 507]]), array([[ 807,  388],
       [ 895,  372],
       [1011,  466],
       [ 921,  488]])]


In [ ]:
while True:
    available_slot=len(parking_lot_coords)

    ret, frame = camera.read()

    if not ret:

        print("[ERROR] Failed to initialize camera.")
        cv2.destroyAllWindows()
        break

    if ret:
        frame=cv2.resize(frame,(1300,650))
        for i in range(len(parking_lot_coords)):

            cv2.polylines(frame,[np.array(parking_lot_coords[i],np.int32)],True,(0,255,0),2)


        results = model.predict(frame)
        result=results[0]
        for box in result.boxes:
            class_id = box.cls[0].item()
            if class_id==0.0:
                cord = box.xyxy[0].tolist()
                cord = [round(x) for x in cord]
                cx=int(cord[0]+cord[2])//2
                cy=int(cord[1]+cord[3])//2
                point=(cx,cy)

                for arr in parking_lot_coords:
                    if cv2.pointPolygonTest(np.array(arr,np.int32),((cx,cy)),False)>=0:
                        available_slot-=1

                        cv2.polylines(frame,[np.array(arr,np.int32)],True,(0,0,255),2)

        print(available_slot)
        cv2.putText(frame,"Available Slots:"+str(available_slot),(50,25),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)


        cv2.imshow("Camera", frame)

        key=cv2.waitKey(1)

        if key%256 == 27:

            # ESC

            print("[INFO] Camera terminated.")
            cv2.destroyAllWindows()
            break

[ERROR] Failed to initialize camera.


In [15]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [16]:
import numpy as np
import cv2

In [17]:
camera = cv2.VideoCapture("parking1.mp4")

In [24]:
from ultralytics import YOLO

from IPython.display import display, Image

In [25]:
model=YOLO(r'yolov8n.pt')

In [26]:
file = open("/content/slot_coordinates.txt.txt")

print("[INFO] Loading parking coordinates ...")

lines = file.readlines()

lines = [line.strip() for line in lines]

total_parking_lots = len(lines)

parking_lot_coords = list()

for i in range(len(lines)):

    coords = lines[i].split()
    top_left = (int(coords[0]), int(coords[1]))
    top_right = (int(coords[2]), int(coords[3]))
    bottom_right = (int(coords[4]), int(coords[5]))
    bottom_left = (int(coords[6]), int(coords[7]))
    coord = np.array([top_left,top_right, bottom_right,bottom_left])
    parking_lot_coords.append(coord)
print(parking_lot_coords)

[INFO] Loading parking coordinates ...
[array([[ 53, 438],
       [133, 441],
       [105, 571],
       [ 23, 568]]), array([[109, 443],
       [205, 436],
       [187, 570],
       [ 89, 567]]), array([[181, 429],
       [287, 428],
       [277, 561],
       [167, 569]]), array([[246, 436],
       [350, 426],
       [365, 557],
       [259, 560]]), array([[331, 424],
       [424, 419],
       [455, 550],
       [356, 564]]), array([[406, 420],
       [497, 410],
       [545, 534],
       [446, 548]]), array([[475, 418],
       [579, 408],
       [640, 524],
       [530, 545]]), array([[540, 405],
       [646, 395],
       [723, 518],
       [612, 533]]), array([[605, 394],
       [715, 388],
       [801, 502],
       [691, 522]]), array([[674, 398],
       [776, 388],
       [877, 487],
       [767, 511]]), array([[737, 385],
       [831, 374],
       [931, 492],
       [859, 507]]), array([[ 807,  388],
       [ 895,  372],
       [1011,  466],
       [ 921,  488]])]


In [28]:
while True:
    available_slot=len(parking_lot_coords)

    ret, frame = camera.read()

    if not ret:

        print("[ERROR] Failed to initialize camera.")
        cv2.destroyAllWindows()
        break

    if ret:
        frame=cv2.resize(frame,(1300,650))
        for i in range(len(parking_lot_coords)):

            cv2.polylines(frame,[np.array(parking_lot_coords[i],np.int32)],True,(0,255,0),2)


        results = model.predict(frame)
        result=results[0]
        for box in result.boxes:
            class_id = box.cls[0].item()
            if class_id==0.0:
                cord = box.xyxy[0].tolist()
                cord = [round(x) for x in cord]
                cx=int(cord[0]+cord[2])//2
                cy=int(cord[1]+cord[3])//2
                point=(cx,cy)

                for arr in parking_lot_coords:
                    if cv2.pointPolygonTest(np.array(arr,np.int32),((cx,cy)),False)>=0:
                        available_slot-=1

                        cv2.polylines(frame,[np.array(arr,np.int32)],True,(0,0,255),2)

        print(available_slot)
        cv2.putText(frame,"Available Slots:"+str(available_slot),(50,25),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)


        cv2.imshow("Camera", frame)

        key=cv2.waitKey(1)

        if key%256 == 27:

            # ESC

            print("[INFO] Camera terminated.")
            cv2.destroyAllWindows()
            break

[ERROR] Failed to initialize camera.


In [29]:
import cv2

# Open the input video file
input_file = 'output_video.mp4'
cap = cv2.VideoCapture(input_file)

# Get the original video's properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the output video file
output_file = 'final_output_video.mp4'
output_fps = 10.0

# Create a VideoWriter object to save the output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
out = cv2.VideoWriter(output_file, fourcc, output_fps, (width, height))

# Variables for frame duplication
frame_count = 0
frame_duplicate = int(fps / output_fps)

# Read and write frames until the end of the video
while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1

    # Duplicate frames
    for _ in range(frame_duplicate):
        out.write(frame)

# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()